In [8]:
print("Ok")

Ok


In [9]:
%pwd

'/Users/sharveshsakthivel/Documents/Projects/Medical-Chatbot/research'

In [10]:

import os
os.chdir("../")

In [11]:
%pwd

'/Users/sharveshsakthivel/Documents/Projects/Medical-Chatbot'

In [12]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [14]:
extracted_data=load_pdf_file(data='/Users/sharveshsakthivel/Documents/Projects/Medical-Chatbot/Data')

In [15]:
# extracted_data

In [16]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [17]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [18]:
# text_chunks

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings

In [20]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [43]:
import sentence_transformers
import huggingface_hub

print(sentence_transformers.__version__)
print(huggingface_hub.__version__)


3.4.1
0.29.1


In [50]:
embeddings = download_hugging_face_embeddings()

In [51]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [26]:
# query_result

In [27]:
from dotenv import load_dotenv
load_dotenv()

True

In [111]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
GEMINIAI_API_KEY=os.environ.get("GEMINI_API_KEY")

In [114]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "llmapp"

pc.delete_index("llmapp")
pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

Error while installing plugin inference: property 'inference' of 'PineconeGRPC' object has no setter
Traceback (most recent call last):
  File "/opt/anaconda3/envs/llmapp/lib/python3.11/site-packages/pinecone_plugin_interface/actions/installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: property 'inference' of 'PineconeGRPC' object has no setter


{
    "name": "llmapp",
    "metric": "cosine",
    "host": "llmapp-offawuh.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [101]:
import google.generativeai as genai


In [115]:
genai.configure(api_key="GEMINI_API_KEY")

In [122]:
model = genai.GenerativeModel("gemini-2.0-flash")

response = model.generate_content("Explain how AI works")
print(response.text)


InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

In [118]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINIAI_API_KEY"] = "GEMINI_API_KEY"

In [68]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

Error while installing plugin inference: property 'inference' of 'Pinecone' object has no setter
Traceback (most recent call last):
  File "/opt/anaconda3/envs/llmapp/lib/python3.11/site-packages/pinecone_plugin_interface/actions/installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: property 'inference' of 'Pinecone' object has no setter


In [69]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

Error while installing plugin inference: property 'inference' of 'Pinecone' object has no setter
Traceback (most recent call last):
  File "/opt/anaconda3/envs/llmapp/lib/python3.11/site-packages/pinecone_plugin_interface/actions/installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: property 'inference' of 'Pinecone' object has no setter


In [72]:
docsearch

In [73]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [74]:
retrieved_docs = retriever.invoke("What is Acne?")

In [75]:
retrieved_docs

[Document(id='d0cea3aa-3e70-4699-a948-9df028a98eaa', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': '/Users/sharveshsakthivel/Documents/Projects/Medical-Chatbot/Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='36146ad7-0873-4c5c-a34a-5f7c20c8ae44', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': '/Users/sharveshsakthivel/Documents/Projects/Medical-Chatbot/Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\ng

In [107]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [110]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [108]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [109]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [91]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}